## Importing modules

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import math
from sklearn.model_selection import GridSearchCV
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Set random seed for reproducibility
np.random.seed(0)

In [2]:
# declaring global font size for plots:
plt.figure()
plt.rcParams.update({'font.size':20})

<Figure size 640x480 with 0 Axes>

## Loading the dataset

In [3]:
# Load the data
df = pd.read_csv('../datasets/complete_data/df.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 37 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   start MTU (UTC)                                                                8760 non-null   object 
 1   Day-ahead Price [EUR/MWh] BZN|NO1                                              8760 non-null   float64
 2   Day-ahead Price [EUR/MWh] BZN|NO3                                              8760 non-null   float64
 3   Day-ahead Price [EUR/MWh] BZN|NO5                                              8760 non-null   float64
 4   Day-ahead Price [EUR/MWh] BZN|SE3                                              8760 non-null   float64
 5   Actual Total Load [MW] - BZN|NO5                                               8760 non-null   float64
 6   Hydro Run-of-river and p

The datatype of the datetime column "start MTU (UTC)" is object, so we need to convert it into a datetime object. We then need to set the "start MTU (UTC)" column as the index, which is required for time series analysis.

In [4]:
# Convert the date column to datetime
df['start MTU (UTC)'] = pd.to_datetime(df['start MTU (UTC)'])

# Set the date column as the index
df.set_index('start MTU (UTC)', inplace=True)


## Splitting the data

In [5]:
# Split the data into training, validation, and test sets
# shuffle is set to false so that the order of the data is maintained
train, test_valid = train_test_split(df, test_size=0.2, shuffle=False)
valid, test = train_test_split(test_valid, test_size=0.5, shuffle=False)

# Print the shapes of the train, valid, and test sets
print("Train set shape:", train.shape)
print("Validation set shape:", valid.shape)
print("Test set shape:", test.shape)


Train set shape: (7008, 36)
Validation set shape: (876, 36)
Test set shape: (876, 36)


## Creating a lagged dataset with 24 lag features for each input feature

When predicting time series, it is important to consider the temporal relationship between observations. Since the observations over time are not independent of each other, we cannot simply use all of the data to train our model as we would in a typical machine learning problem. Instead, we typically create a "lagged" dataset that contains the predictors and the target variable for each time step. This lagged dataset is used to train our model.

To create a lagged dataset, we shift the target variable forward by one or more time steps, using the resulting series as the target variable for that time step. We also include the values of the predictors at the previous time steps as features. This way, the model can learn to use past predictor values to predict future target variable values.

For instance, for hourly data and a prediction horizon of 24 hours, we would create a lagged dataset where each row contains the predictors and the target variable for a specific hour. The target variable for each row would be the value 24 hours in the future, and the predictors for each row would be the values of the predictors for the previous 24 hours. This lagged dataset would then be used to train our model to make predictions for the next 24 hours.

By shifting the target feature forward by one period, we create a dataset where each observation includes the past observations up to the previous period and the target variable value for the next period.

In [6]:
# Function to create lagged dataset wit 24 lag features for each input feature
def create_lagged_dataset(df):
    # creating a copy of the dataframe
    lagged_df = df.copy()
    
    # Adding lagged features for target variable
    lagged_df['Day-ahead Price [EUR/MWh] BZN|NO1'] = lagged_df['Day-ahead Price [EUR/MWh] BZN|NO1'].shift(-1)
    
    # Dropping the last row containing NaN values
    lagged_df.dropna(inplace=True)
    
    # Creating a dataframe with lagged features with 24 steps for each of the original features
    lagged_df = pd.concat([lagged_df.shift(i) for i in range(24)], axis=1)
    
    # Removing the NaN rows that have been created in the beginning of the dataset
    lagged_df.dropna(inplace=True)
    
    return lagged_df

In [7]:
# creating lagged dataset for train and test dataset
lagged_train = create_lagged_dataset(train)
lagged_valid = create_lagged_dataset(valid)
lagged_test = create_lagged_dataset(test)


In [8]:
# separating the target feature and the input features
train_y = lagged_train['Day-ahead Price [EUR/MWh] BZN|NO1']
train_x = lagged_train.drop(['Day-ahead Price [EUR/MWh] BZN|NO1'], axis=1)

valid_y = lagged_valid['Day-ahead Price [EUR/MWh] BZN|NO1']
valid_x = lagged_valid.drop(['Day-ahead Price [EUR/MWh] BZN|NO1'], axis=1)

test_y = lagged_test['Day-ahead Price [EUR/MWh] BZN|NO1']
test_x = lagged_test.drop(['Day-ahead Price [EUR/MWh] BZN|NO1'], axis=1)

#print the shapes of the datasets
print("Train Data Shape: ", train_x.shape, train_y.shape)
print("Validation Data Shape: ", valid_x.shape, valid_y.shape)
print("Test Data Shape: ", test_x.shape, test_y.shape)

Train Data Shape:  (6984, 840) (6984, 24)
Validation Data Shape:  (852, 840) (852, 24)
Test Data Shape:  (852, 840) (852, 24)


## Training the models

In [ ]:
# start timer
start_time = time.time()

# Create a random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=0)

# Fit the model on the training data
rf.fit(train_x, train_y)
# stop timer
end_time = time.time()

# Make predictions on the test data
pred_y = rf.predict(test_x)


#print the run time
print("Runtime: {:.2f} seconds".format(end_time - start_time))

# Calculate the mean absolute error (MAE) between the predicted and actual values
mae = mean_absolute_error(test_y, pred_y)
print("Mean absolute error:            ", mae)

# Calculate the mean absolute percentage error (MAE) between the predicted and actual values
mape = mean_absolute_percentage_error(test_y, pred_y)
print("Mean absolute percentage error: ", mape)

# Calculating the mean squared error (MSE) between the predicted and actual values
mse = mean_squared_error(test_y, pred_y)
print("Mean squared error:             ", mse)

# Calculating the root mean squared error (RMSE) between the predicted and actual values
rmse = math.sqrt(mse)
print("Rood mean squared error:        ", rmse)

## Plotting mean of predicted vs mean of actual values for the base rf model:

In [ ]:
# Set the figure size
plt.figure(figsize=(18, 7))

# Plot the mean of the actual and predicted values
plt.plot(test_y.index, np.mean(test_y, axis=1), label='Actual')
plt.plot(test_y.index, np.mean(pred_y, axis=1), label='Predicted')

# Set the title and axis labels
plt.title('Mean of Actual vs mean of Predicted Day-ahead Price [EUR/MWh] BZN|NO1')
plt.xlabel('Date')
plt.ylabel('Price [EUR/MWh]')

# Rotate the x-axis labels
plt.xticks(rotation=30)

# Add a legend
plt.legend()

# Show the plot
plt.show()


### NB! this is direct copy, needs to be rewritten!
In terms of MAPE, a commonly used benchmark is a value less than 10%. However, this can vary depending on the specific industry and application. For example, in finance, a MAPE less than 2% may be considered good.

For RMSE, a good value depends on the scale of the data being analyzed. A common practice is to compare the RMSE to the range of the target variable. As a rough guideline, an RMSE that is less than 10% of the range of the target variable can be considered good.

## hyper parameter tuning

To handle the time series nature of the data, rolling origin cross-validation is used, which involves training the model on a rolling window of historical data and testing it on the next observation. The primary benefit of using the rolling origin cross-validation method is that it maintains the temporal order of the data, which is essential for accurate time series forecasting. This approach ensures that the model is evaluated on data that is more similar to the future data it will be predicting.

A random search is performed on a dictionary of hyperparameters to search for the best combination of hyperparameters for the random forest regressor model. The hyperparameters tuned in this process include the number of trees in the forest (n_estimators), the maximum depth of each tree (max_depth), the minimum number of samples required to split an internal node (min_samples_split), the minimum number of samples required to be at a leaf node (min_samples_leaf), and the maximum number of features to consider when looking for the best split (max_features).

In [ ]:
# start timer
start_time = time.time()

# Create a random forest regressor object
rf = RandomForestRegressor(random_state=0)

# Define a dictionary of hyperparameters to search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
}

n_folds = 5

# Initialize lists to store the performance metrics
rmse_scores = []
mae_scores = []
mape_scores = []
mse_scores = []



# Perform rolling origin cross-validation
for i in range(n_folds, len(valid_y)):
    # Split the data into training and validation sets
    train_y = valid_y.iloc[i-n_folds:i]
    train_x = valid_x.iloc[i-n_folds:i]
    val_y = valid_y.iloc[i:i+1]
    val_x = valid_x.iloc[i:i+1]

    # Perform a random search on the training data
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=n_folds, n_jobs=-1, random_state=0, scoring='neg_mean_squared_error')
    random_search.fit(train_x, train_y)

    # Extract the best model from the random search
    best_rf_model = random_search.best_estimator_

    # Make predictions on the validation data using the best model
    val_predictions = best_rf_model.predict(val_x)

    # Calculate the performance metrics for the current fold
    rmse_scores.append(math.sqrt(mean_squared_error(val_y, val_predictions)))
    mae_scores.append(mean_absolute_error(val_y, val_predictions))
    mape_scores.append(mean_absolute_percentage_error(val_y, val_predictions))
    mse_scores.append(mean_squared_error(val_y, val_predictions))

# stop timer
end_time = time.time()

# Print the runtime
print("Runtime: {:.2f} seconds".format(end_time - start_time))

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Print the average performance metrics over all folds
print("Average Mean Absolute Error (MAE):", round(np.mean(mae_scores), 2))
print("Average Mean Absolute Percentage Error (MAPE):", round(np.mean(mape_scores), 2), "%")
print("Average Mean Squared Error (MSE):", round(np.mean(mse_scores), 2))
print("Average Root Mean Squared Error (RMSE):", round(np.mean(rmse_scores), 2))


## Plotting mean of predicted vs mean of actual values for the best rf model with validation data:

In [ ]:
# Define the optimal hyperparameters found through grid search
optimal_params = random_search.best_params_

# Create a random forest regressor object with the optimal hyperparameters
rf = RandomForestRegressor(**optimal_params, random_state=0)

# Train the model on the validation data
rf.fit(valid_x, valid_y)

# Make predictions on the validation data using the trained model
val_predictions = rf.predict(valid_x)

# Set the figure size
plt.figure(figsize=(18, 10))

# Plot the mean of the actual and predicted values
plt.plot(test_y.index, np.mean(test_y, axis=1), label='Actual')
plt.plot(test_y.index, np.mean(test_pred_y, axis=1), label='Predicted')

# Set the title and axis labels
plt.title('Mean of Actual vs mean of Predicted Day-ahead Price [EUR/MWh] BZN|NO1')
plt.xlabel('Date')
plt.ylabel('Price [EUR/MWh]')

# Rotate the x-axis labels
plt.xticks(rotation=30)

# Add a legend
plt.legend()

# Show the plot
plt.show()

In order to assess the performance of the model with the optimal hyperparameters on the test data, we must first train the model on the combined training and validation data using the tuned hyperparameters obtained from the cross-validation grid search. After training the model, we can use it to make predictions on the test data and measure its performance using the same metrics (MAE, MAPE, MSE, and RMSE) that were used to evaluate its performance on the validation data.

In [ ]:
# Combine the training and validation data
train_x = pd.concat([train_x, valid_x])
train_y = pd.concat([train_y, valid_y])

# Define the optimal hyperparameters found through grid search
optimal_params = random_search.best_params_

# Create a random forest regressor object with the optimal hyperparameters
rf = RandomForestRegressor(**optimal_params, random_state=0)

# Train the model on the combined training and validation data
rf.fit(train_x, train_y)

# Make predictions on the test data using the trained model
test_pred_y = rf.predict(test_x)

# Evaluate the performance of the model on the test data using the same metrics used for validation data
test_rmse = math.sqrt(mean_squared_error(test_y, test_pred_y))
test_mae = mean_absolute_error(test_y, test_pred_y)
test_mape = mean_absolute_percentage_error(test_y, test_pred_y)
test_mse = mean_squared_error(test_y, test_pred_y)

# Print the performance metrics on the test data
print("Root Mean Squared Error on Test Data:", round(test_rmse, 2))
print("Mean Absolute Error on Test Data:", round(test_mae, 2))
print("Mean Absolute Percentage Error on Test Data:", round(test_mape, 2))
print("Mean Squared Error on Test Data:", round(test_mse, 2))

## Plotting mean of predicted vs mean of actual values for the best rf model with test data:

In [ ]:

# Set the figure size
plt.figure(figsize=(18, 10))

# Plot the mean of the actual and predicted values
plt.plot(test_y.index, np.mean(test_y, axis=1), label='Actual')
plt.plot(test_y.index, np.mean(test_pred_y, axis=1), label='Predicted')

# Set the title and axis labels
plt.title('Mean of Actual vs mean of Predicted Day-ahead Price [EUR/MWh] BZN|NO1')
plt.xlabel('Date')
plt.ylabel('Price [EUR/MWh]')

# Rotate the x-axis labels
plt.xticks(rotation=30)

# Add a legend
plt.legend()

# Show the plot
plt.show()